In [3]:
# !pip install transformers
# !pip install pyyaml
# !pip install google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2

In [5]:
model_name = "Llama3.2-3B-Instruct"
models_path = r"./llama-models/"
transformed_model_path = models_path + model_name + "-transformed"

In [6]:
from transformers import LlamaForCausalLM, AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LlamaForCausalLM.from_pretrained(transformed_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(transformed_model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generate songs
----------

In [7]:
import time

def infer(prompt):
    start_time = time.time()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generate_ids = model.generate(inputs.input_ids, max_length=500, eos_token_id=None)
    response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    elapsed_time = time.time() - start_time
    print(f"\n({elapsed_time:.4f} seconds taken on {model.device})")
    return response[0]

In [8]:
mood = "jazz"
prompt = f"Generate a {mood} music playlist. In 'index. name - artist' format."
response = infer(prompt)
print(response)


(331.6935 seconds taken on cuda:0)
Generate a jazz music playlist. In 'index. name - artist' format. 20 tracks to get you grooving. 
1. So What - Miles Davis
2. Take Five - The Dave Brubeck Quartet
3. My Funny Valentine - Chet Baker
4. Fly Me to the Moon - Frank Sinatra
5. Blue Moon - Billie Holiday
6. The Girl from Ipanema - Stan Getz and Astrud Gilberto
7. I'll Be Seeing You - Billie Holiday
8. Summertime - Louis Armstrong
9. Sway - Dean Martin
10. Moon River - Andy Williams
11. The Nearness of You - Billie Holiday
12. Summertime Blues - Louis Armstrong
13. I've Got You Under My Skin - Frank Sinatra
14. Take the A Train - Duke Ellington
15. Blue Skies - Ella Fitzgerald
16. How High the Moon - Les Paul
17. The Way You Look Tonight - Frank Sinatra
18. I'll Never Smile Again - Frank Sinatra
19. The Lady Is a Tramp - Ella Fitzgerald
20. Ain't Misbehavin' - Fats Waller

This playlist is a mix of jazz standards, classic jazz, and some lesser-known gems. It's designed to take you on a jour

In [9]:
import re

def sanitize_line(line):
    line = re.sub(r"(^[0-9]+(.|)|[*\"])", "", line)
    return line.strip()

def extract_list(text):
    filtered = text.split('\n')
    filtered = filter(lambda line: re.match(r"[0-9]+.+-.+", line), filtered)
    filtered = map(lambda line: sanitize_line(line), filtered)
    return list(filtered)

songs = extract_list(response)
for song in songs: print(song)

So What - Miles Davis
Take Five - The Dave Brubeck Quartet
My Funny Valentine - Chet Baker
Fly Me to the Moon - Frank Sinatra
Blue Moon - Billie Holiday
The Girl from Ipanema - Stan Getz and Astrud Gilberto
I'll Be Seeing You - Billie Holiday
Summertime - Louis Armstrong
Sway - Dean Martin
Moon River - Andy Williams
The Nearness of You - Billie Holiday
Summertime Blues - Louis Armstrong
I've Got You Under My Skin - Frank Sinatra
Take the A Train - Duke Ellington
Blue Skies - Ella Fitzgerald
How High the Moon - Les Paul
The Way You Look Tonight - Frank Sinatra
I'll Never Smile Again - Frank Sinatra
The Lady Is a Tramp - Ella Fitzgerald
Ain't Misbehavin' - Fats Waller
So What - Miles Davis
Take Five - The Dave Brubeck Quartet
My Funny Valentine - Chet Baker
Fly Me to the Moon - Frank Sinatra
Blue Moon - Billie Holiday
The Girl from Ipanema - Stan Get


Find songs on Youtube
--------------

In [8]:
# import yaml

# def read_yaml(yml_path):
#     with open(yml_path, 'r') as file:
#         properties = yaml.safe_load(file)
#     return properties

# properties = read_yaml("secret.yml")
# google_api_key = properties['google_api_key']
# youtube = build('youtube', 'v3', developerKey=google_api_key)

In [9]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import pickle

# Scopes required for YouTube API
SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

def authenticate_youtube():
    creds = None
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "client_secret.json", SCOPES)
            creds = flow.run_local_server(port=0)

        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)

    return build("youtube", "v3", credentials=creds)

youtube = authenticate_youtube()

In [10]:
from googleapiclient.discovery import build

def search_youtube(query, max_results=5):
    request = youtube.search().list(
        part="snippet",
        q=query,
        type="video",
        maxResults=max_results
    )
    response = request.execute()
    
    videos = []
    for item in response.get("items", []):
        video_data = {
            "title": item["snippet"]["title"],
            "channel": item["snippet"]["channelTitle"],
            "description": item["snippet"]["description"],
            "video_id": item['id']['videoId'],
            "video_url": f"https://music.youtube.com/watch?v={item['id']['videoId']}"
        }
        videos.append(video_data)
    
    return videos

videos = []
for song in songs:
    result = search_youtube(song)[0]
    print(f"{song:50}\t{result['video_url']}")
    videos.append(result)

Miles Davis - 'Kind of Blue'                      	https://music.youtube.com/watch?v=FEPFH-gz3wE
John Coltrane - 'My Favorite Things'              	https://music.youtube.com/watch?v=JQvc-Gkwhow
Billie Holiday - 'God Bless the Child'            	https://music.youtube.com/watch?v=bKNtP1zOVHw
Ella Fitzgerald - 'Misty Blue'                    	https://music.youtube.com/watch?v=yZ_24PCfTu4
Louis Armstrong - 'What a Wonderful World'        	https://music.youtube.com/watch?v=rBrd_3VMC3c
Duke Ellington - 'Take the A Train'               	https://music.youtube.com/watch?v=D6mFGy4g_n8
Thelonious Monk - 'Round Midnight'                	https://music.youtube.com/watch?v=IKayR1oqC7w
Charlie Parker - 'Now's the Time'                 	https://music.youtube.com/watch?v=ryNtmkfeJk4
Dizzy Gillespie - 'A Night in Tunisia'            	https://music.youtube.com/watch?v=mkemox0461U
Count Basie - 'April in Paris'                    	https://music.youtube.com/watch?v=wCmcoZktZG4
Miles Davis - 'So What'       

Create playlist
---------

In [13]:
import time
from googleapiclient.errors import HttpError

def create_playlist(youtube, title, description, privacy_status):
    try:
        request = youtube.playlists().insert(
            part="snippet,status",
            body={
                "snippet": {
                    "title": title,
                    "description": description,
                    "tags": ["sample playlist", "API-created"],
                    "defaultLanguage": "en"
                },
                "status": {
                    "privacyStatus": privacy_status  # "public", "private", or "unlisted"
                }
            }
        )
        response = request.execute()
        print(f"Playlist created: {response['id']}")
        return response["id"]
    except HttpError as e:
        print(f"An error occurred: {e}")
        return None

playlist_id = create_playlist(youtube, f"Llama generated playlist-{time.time()} ({mood})", "This is a playlist created with the YouTube API using Llama 3.2.", "unlisted")
print(f"Playlist URL: https://www.youtube.com/playlist?list={playlist_id}")


Playlist created: PLSvM43_JInG1o_lLQXZmZfea2c-BNJV2i
Playlist URL: https://www.youtube.com/playlist?list=PLSvM43_JInG1o_lLQXZmZfea2c-BNJV2i


In [14]:
def add_video_to_playlist(youtube, playlist_id, video_id):
    try:
        request = youtube.playlistItems().insert(
            part="snippet",
            body={
                "snippet": {
                    "playlistId": playlist_id,  # The ID of the playlist
                    "resourceId": {
                        "kind": "youtube#video",  # Specify the kind of resource
                        "videoId": video_id      # The ID of the video to add
                    }
                }
            }
        )
        response = request.execute()
        print(f"Video added to playlist: {video_id}")
    except HttpError as e:
        print(f"An error occurred: {e}")

for video in videos:
    add_video_to_playlist(youtube, playlist_id, video['video_id'])

Video added to playlist: FEPFH-gz3wE
Video added to playlist: JQvc-Gkwhow
Video added to playlist: bKNtP1zOVHw
Video added to playlist: yZ_24PCfTu4
Video added to playlist: rBrd_3VMC3c
Video added to playlist: D6mFGy4g_n8
Video added to playlist: IKayR1oqC7w
Video added to playlist: ryNtmkfeJk4
Video added to playlist: mkemox0461U
Video added to playlist: wCmcoZktZG4
Video added to playlist: ylXk1LBvIqU
Video added to playlist: KwIC6B_dvW4
Video added to playlist: -DGY9HvChXk
Video added to playlist: yZ_24PCfTu4
Video added to playlist: rGGZ3zMOUhg
Video added to playlist: RFUSD23ZXEw
